<a href="https://colab.research.google.com/github/rajveer43/titan_transformer/blob/master/Google_Titan_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titan implementation

## Titan Model Implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.depthwise = nn.Conv1d(
            in_channels, in_channels, kernel_size,
            padding=padding, groups=in_channels
        )
        self.pointwise = nn.Conv1d(in_channels, out_channels, 1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class GatingMechanism(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gate_proj = nn.Linear(dim, dim)
        self.transform_proj = nn.Linear(dim, dim)

    def forward(self, x):
        gate = torch.sigmoid(self.gate_proj(x))
        transformed = self.transform_proj(x)
        return gate * transformed

class TitanAttention(nn.Module):
    def __init__(self, dim, num_heads=8, head_dim=64, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.scale = head_dim ** -0.5

        # Projections for Q, K, V
        self.q_proj = nn.Linear(dim, num_heads * head_dim)
        self.k_proj = nn.Linear(dim, num_heads * head_dim)
        self.v_proj = nn.Linear(dim, num_heads * head_dim)

        # Depthwise separable convolutions
        self.q_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )
        self.k_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )
        self.v_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )

        self.out_proj = nn.Linear(num_heads * head_dim, dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        batch_size, seq_len, dim = x.shape

        # Project and apply SiLU activation
        q = F.silu(self.q_proj(x))
        k = F.silu(self.k_proj(x))
        v = F.silu(self.v_proj(x))

        # Reshape for depthwise conv
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Apply depthwise separable convolutions
        q = self.q_conv(q).transpose(1, 2)
        k = self.k_conv(k).transpose(1, 2)
        v = self.v_conv(v).transpose(1, 2)

        # Reshape for multi-head attention
        q = q.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # L2 normalize queries and keys
        q = F.normalize(q, p=2, dim=-1)
        k = F.normalize(k, p=2, dim=-1)

        # Compute attention scores
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        if mask is not None:
            # Properly reshape mask for broadcasting
            # mask shape: [batch_size, seq_len] -> [batch_size, 1, 1, seq_len]
            mask = mask.unsqueeze(1).unsqueeze(2)
            attn = attn.masked_fill(mask == 0, float('-inf'))

        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)

        # Apply attention to values
        out = torch.matmul(attn, v)

        # Reshape and project output
        out = out.transpose(1, 2).reshape(batch_size, seq_len, -1)
        out = self.out_proj(out)

        return out

class TitanBlock(nn.Module):
    def __init__(self, dim, num_heads=8, head_dim=64, mlp_ratio=4, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = TitanAttention(dim, num_heads, head_dim, dropout)
        self.norm2 = nn.LayerNorm(dim)

        mlp_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.SiLU(),
            GatingMechanism(mlp_dim),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask=None):
        # Residual connection for attention
        x = x + self.attn(self.norm1(x), mask)
        # Residual connection for MLP
        x = x + self.mlp(self.norm2(x))
        return x

class TitanTransformer(nn.Module):
    def __init__(
        self,
        dim,
        depth,
        num_heads=8,
        head_dim=64,
        mlp_ratio=4,
        dropout=0.1
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TitanBlock(
                dim=dim,
                num_heads=num_heads,
                head_dim=head_dim,
                mlp_ratio=mlp_ratio,
                dropout=dropout
            )
            for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(dim)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# Example usage
def create_titan_model(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12,
    num_heads=8,
    head_dim=64,
    mlp_ratio=4,
    dropout=0.1
):
    class TitanModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, dim)
            self.pos_embedding = nn.Parameter(torch.zeros(1, max_seq_length, dim))
            self.transformer = TitanTransformer(
                dim=dim,
                depth=depth,
                num_heads=num_heads,
                head_dim=head_dim,
                mlp_ratio=mlp_ratio,
                dropout=dropout
            )

        def forward(self, x, mask=None):
            # Add positional embeddings
            x = self.embedding(x)
            x = x + self.pos_embedding[:, :x.size(1), :]

            # Apply transformer
            x = self.transformer(x, mask)
            return x

    return TitanModel()

In [2]:
# Create a model
model = create_titan_model(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12
)

# Example input (batch_size=2, seq_length=128)
x = torch.randint(0, 50000, (2, 128))
mask = torch.ones(2, 128)

# Forward pass
output = model(x, mask)

In [3]:
print(output)

tensor([[[-0.0967, -1.5880,  0.4721,  ..., -2.2226,  1.4096, -1.2016],
         [ 0.3354, -0.0147,  1.5741,  ..., -1.1080,  1.0628,  0.5618],
         [ 0.6820,  1.6194,  1.6092,  ..., -1.7524, -0.3709,  0.3478],
         ...,
         [ 0.4796, -0.3505,  0.9744,  ..., -0.2081,  0.1899,  0.2010],
         [ 1.0270, -0.3352,  1.5500,  ..., -1.2115, -0.6697, -0.1870],
         [-0.7622, -1.3119,  1.2983,  ..., -0.9509, -0.5621,  0.1794]],

        [[ 1.4440, -0.6268, -0.6059,  ...,  0.3159, -1.1959, -0.0720],
         [ 0.1174,  0.2564,  0.0666,  ...,  0.2098, -0.0316,  0.4069],
         [-0.3333,  0.6420,  1.4212,  ..., -0.6555, -0.0132, -0.0676],
         ...,
         [ 2.0356,  0.2566, -0.3156,  ...,  0.4360,  0.4951, -2.2695],
         [ 0.5867, -0.8586,  0.8307,  ...,  0.3732,  0.9553,  0.5013],
         [-0.5009, -1.7895,  2.6095,  ...,  1.9907, -1.1462,  0.2110]]],
       grad_fn=<NativeLayerNormBackward0>)


## using the huggingface tokenizer

In [4]:
import torch
import torch.nn.functional as F
from typing import List, Optional, Tuple
from transformers import PreTrainedTokenizerFast
from transformers import GPT2Tokenizer

class TitanTextGenerator:
    def __init__(
        self,
        model: 'TitanModel',
        tokenizer: Optional[PreTrainedTokenizerFast] = None,
        max_length: int = 1024,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self.model = model.to(device)
        self.model.eval()  # Set to evaluation mode
        self.device = device
        self.max_length = max_length

        # Initialize tokenizer if not provided
        if tokenizer is None:
            self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            self.tokenizer = tokenizer

    def preprocess_text(self, text: str) -> Tuple[torch.Tensor, torch.Tensor]:
        # Tokenize input text
        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encodings['input_ids'].to(self.device)
        attention_mask = encodings['attention_mask'].to(self.device)

        return input_ids, attention_mask

    @torch.no_grad()
    def generate_next_token(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        temperature: float = 1.0,
        top_k: int = 50,
        top_p: float = 0.9
    ) -> torch.Tensor:
        # Get model output
        outputs = self.model(input_ids, attention_mask)

        # Get logits for the next token (last position)
        next_token_logits = outputs[:, -1, :]

        # Apply temperature
        next_token_logits = next_token_logits / temperature

        # Apply top-k filtering
        if top_k > 0:
            values, indices = torch.topk(next_token_logits, top_k)
            min_values = values[:, -1].unsqueeze(-1).expand_as(next_token_logits)
            next_token_logits = torch.where(
                next_token_logits < min_values,
                torch.ones_like(next_token_logits) * float('-inf'),
                next_token_logits
            )

        # # Apply top-p (nucleus) filtering
        # if top_p < 1.0:
        #     # Sort logits in descending order
        #     sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
        #     cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        #     # Create a mask for tokens to remove
        #     sorted_indices_to_remove = cumulative_probs > top_p

        #     # Shift the indices to the right to keep also the first token above the threshold
        #     sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        #     sorted_indices_to_remove[..., 0] = 0

        #     # Scatter sorted indices to original positions
        #     indices_to_remove = sorted_indices_to_remove.scatter(
        #         dim=1,
        #         index=sorted_indices,
        #         src=sorted_indices_to_remove
        #     )
        #     next_token_logits = next_token_logits.masked_fill(indices_to_remove, float('-inf'))

        # Apply top-p (nucleus) filtering
        if top_p < 1.0:
            # Sort logits in descending order
            sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

            # Mask tokens with cumulative probability above top_p
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0  # Keep at least one token

            # Scatter mask back to logits
            mask = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
            next_token_logits = next_token_logits.masked_fill(mask, float('-inf'))


        # Sample from the filtered distribution
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        return next_token

    def generate(
        self,
        text: str,
        max_new_tokens: int = 50,
        temperature: float = 1.0,
        top_k: int = 50,
        top_p: float = 0.9,
        early_stopping: bool = True
    ) -> str:
        # Preprocess input text
        input_ids, attention_mask = self.preprocess_text(text)

        generated_tokens = []

        for _ in range(max_new_tokens):
            # Generate next token
            next_token = self.generate_next_token(
                input_ids,
                attention_mask,
                temperature,
                top_k,
                top_p
            )

            # Append the new token
            generated_tokens.append(next_token.item())

            # Update input_ids and attention_mask
            input_ids = torch.cat([input_ids, next_token], dim=1)
            attention_mask = torch.cat([
                attention_mask,
                torch.ones((1, 1), device=self.device)
            ], dim=1)

            # Check for EOS token
            if early_stopping and next_token.item() == self.tokenizer.eos_token_id:
                break

            # Check if we've exceeded max_length
            if input_ids.size(1) >= self.max_length:
                break

        # Decode and return the complete text
        full_text = self.tokenizer.decode(
            input_ids[0].tolist(),
            skip_special_tokens=True
        )

        return full_text

# Example usage function
def create_text_generator(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12,
    device='cuda' if torch.cuda.is_available() else 'cpu'
):
    # Create the base model
    model = create_titan_model(
        vocab_size=vocab_size,
        max_seq_length=max_seq_length,
        dim=dim,
        depth=depth
    )

    # Create the generator
    generator = TitanTextGenerator(
        model=model,
        max_length=max_seq_length,
        device=device
    )

    return generator

In [5]:
# Create the generator
generator = create_text_generator(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12
)

# Generate text
input_text = "MS thoni is world's best "
generated_text = generator.generate(
    text=input_text,
    max_new_tokens=50,
    temperature=0.3,  # Lower for more focused/deterministic output
    top_k=50,        # Number of highest probability vocabulary tokens to keep
    top_p=0.9        # Cumulative probability threshold for nucleus sampling
)

print(generated_text)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

MS thoni is world's best D not..) canial� W bJ f�ivend�A&A& we�A& weokar00olon int�A& e*"�ide�rell W b their p%re


## using custom tokenizer

In [6]:
import torch
import torch.nn as nn
from collections import Counter
from typing import List, Dict, Tuple
import re
import numpy as np
from torch.nn.utils.rnn import pad_sequence

class CustomTokenizer:
    def __init__(
        self,
        vocab_size: int = 50000,
        min_freq: int = 2,
        special_tokens: List[str] = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
    ):
        self.vocab_size = vocab_size
        self.min_freq = min_freq
        self.special_tokens = special_tokens

        # Initialize special token IDs
        self.pad_token_id = 0
        self.unk_token_id = 1
        self.bos_token_id = 2
        self.eos_token_id = 3

        # Initialize vocabularies
        self.token2idx: Dict[str, int] = {token: idx for idx, token in enumerate(special_tokens)}
        self.idx2token: Dict[int, str] = {idx: token for idx, token in enumerate(special_tokens)}
        self.vocab_size_current = len(special_tokens)

        # Regex for tokenization
        self.token_pattern = re.compile(r'\w+|[^\w\s]')

    def train_from_texts(self, texts: List[str]) -> None:
        """Train tokenizer on a list of texts."""
        # Count word frequencies
        word_counts = Counter()

        for text in texts:
            tokens = self._basic_tokenize(text)
            word_counts.update(tokens)

        # Filter by minimum frequency and vocab size
        filtered_tokens = [
            token for token, count in word_counts.most_common()
            if count >= self.min_freq and token not in self.special_tokens
        ]

        # Add tokens to vocabulary up to vocab_size
        remaining_space = self.vocab_size - len(self.special_tokens)
        for token in filtered_tokens[:remaining_space]:
            self.token2idx[token] = self.vocab_size_current
            self.idx2token[self.vocab_size_current] = token
            self.vocab_size_current += 1

    def _basic_tokenize(self, text: str) -> List[str]:
        """Basic tokenization into words and punctuation."""
        return self.token_pattern.findall(text.lower())

    def encode(self, text: str, add_special_tokens: bool = True) -> List[int]:
        """Encode text to token ids."""
        tokens = self._basic_tokenize(text)

        ids = []
        if add_special_tokens:
            ids.append(self.bos_token_id)

        for token in tokens:
            ids.append(self.token2idx.get(token, self.unk_token_id))

        if add_special_tokens:
            ids.append(self.eos_token_id)

        return ids

    def decode(self, ids: List[int], skip_special_tokens: bool = True) -> str:
        """Decode token ids back to text."""
        tokens = []
        for idx in ids:
            token = self.idx2token.get(idx, "<UNK>")
            if skip_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        return " ".join(tokens)

    def save_vocab(self, path: str) -> None:
        """Save vocabulary to file."""
        with open(path, 'w', encoding='utf-8') as f:
            for token, idx in sorted(self.token2idx.items(), key=lambda x: x[1]):
                f.write(f"{token}\t{idx}\n")

    def load_vocab(self, path: str) -> None:
        """Load vocabulary from file."""
        self.token2idx.clear()
        self.idx2token.clear()
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                token, idx = line.strip().split('\t')
                idx = int(idx)
                self.token2idx[token] = idx
                self.idx2token[idx] = token
        self.vocab_size_current = len(self.token2idx)

class CustomEmbedding(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embedding_dim: int,
        pad_idx: int = 0,
        max_norm: float = None
    ):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=pad_idx,
            max_norm=max_norm
        )
        self.embedding_dim = embedding_dim

        # Initialize embeddings using Xavier uniform initialization
        nn.init.xavier_uniform_(self.embedding.weight)
        with torch.no_grad():
            self.embedding.weight[pad_idx].fill_(0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.embedding(x)

class TextDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        texts: List[str],
        tokenizer: CustomTokenizer,
        max_length: int = 1024
    ):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Tokenize all texts
        self.encoded_texts = [
            self.tokenizer.encode(text)[:max_length] for text in texts
        ]

    def __len__(self) -> int:
        return len(self.encoded_texts)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        tokens = self.encoded_texts[idx]

        # Create input and target sequences for language modeling
        input_ids = torch.tensor(tokens[:-1], dtype=torch.long)
        target_ids = torch.tensor(tokens[1:], dtype=torch.long)

        return input_ids, target_ids

def create_dataloader(
    texts: List[str],
    tokenizer: CustomTokenizer,
    batch_size: int = 32,
    max_length: int = 1024,
    shuffle: bool = True
) -> torch.utils.data.DataLoader:
    dataset = TextDataset(texts, tokenizer, max_length)

    # Custom collate function to handle variable length sequences
    def collate_fn(batch):
        input_ids, target_ids = zip(*batch)

        # Pad sequences
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        target_ids = pad_sequence(target_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

        # Create attention mask
        attention_mask = (input_ids != tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'target_ids': target_ids,
            'attention_mask': attention_mask
        }

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn
    )

# Example usage
def create_custom_tokenizer_and_embedding(
    texts: List[str],
    vocab_size: int = 50000,
    embedding_dim: int = 512,
    min_freq: int = 2
):
    # Create and train tokenizer
    tokenizer = CustomTokenizer(vocab_size=vocab_size, min_freq=min_freq)
    tokenizer.train_from_texts(texts)

    # Create embedding layer
    embedding = CustomEmbedding(
        vocab_size=tokenizer.vocab_size_current,
        embedding_dim=embedding_dim,
        pad_idx=tokenizer.pad_token_id
    )

    return tokenizer, embedding

In [7]:
class TitanModelWithCustomEmbedding(nn.Module):
    def __init__(
        self,
        embedding_layer: CustomEmbedding,
        max_seq_length: int = 1024,
        depth: int = 12,
        num_heads: int = 8,
        head_dim: int = 64,
        mlp_ratio: int = 4,
        dropout: float = 0.1
    ):
        super().__init__()
        dim = embedding_layer.embedding_dim

        self.embedding = embedding_layer
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_seq_length, dim))

        self.transformer = TitanTransformer(
            dim=dim,
            depth=depth,
            num_heads=num_heads,
            head_dim=head_dim,
            mlp_ratio=mlp_ratio,
            dropout=dropout
        )

        # Add final projection for token prediction
        self.output_projection = nn.Linear(dim, embedding_layer.embedding.num_embeddings)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None):
        # Get embeddings
        x = self.embedding(input_ids)

        # Add positional embeddings
        x = x + self.pos_embedding[:, :x.size(1), :]

        # Apply transformer
        x = self.transformer(x, attention_mask)

        # Project to vocabulary
        logits = self.output_projection(x)

        return logits

# Example usage
def create_complete_model(texts: List[str], vocab_size: int = 50000, embedding_dim: int = 512):
    # Create tokenizer and embedding
    tokenizer, embedding = create_custom_tokenizer_and_embedding(
        texts=texts,
        vocab_size=vocab_size,
        embedding_dim=embedding_dim
    )

    # Create model
    model = TitanModelWithCustomEmbedding(
        embedding_layer=embedding,
        max_seq_length=1024,
        depth=12
    )

    return model, tokenizer

In [8]:
# Example texts for training
texts = [
    "Here is an example text.",
    "Another example for training.",
    # ... add more texts
]

# Create model and tokenizer
model, tokenizer = create_complete_model(texts)

# Create dataloader
dataloader = create_dataloader(
    texts=texts,
    tokenizer=tokenizer,
    batch_size=32
)

# Example of processing a single text
text = "Here is a test sentence."
input_ids = torch.tensor([tokenizer.encode(text)], dtype=torch.long)
attention_mask = (input_ids != tokenizer.pad_token_id).float()

# Get model output
outputs = model(input_ids, attention_mask)

In [9]:
print(outputs)

tensor([[[-0.8365,  0.1644, -0.6665, -0.4761, -0.0949,  0.0239],
         [-0.7550,  0.0202, -0.7357, -0.5227, -0.6834, -0.0340],
         [-0.9247,  0.0017, -0.5474, -0.7903, -0.6522,  0.1734],
         [-0.7042, -0.1944, -0.3851, -0.4764, -0.4044,  0.4866],
         [-1.1108, -0.0616, -0.6679, -0.6358, -0.3751,  0.4013],
         [-0.7235, -0.0043, -0.3396, -0.5388, -0.7040,  0.1414],
         [-1.0377, -0.1011, -0.3936, -0.5497, -0.3725,  0.3084],
         [-0.7377,  0.1324, -0.0662, -0.2708, -0.4715,  0.3859]]],
       grad_fn=<ViewBackward0>)


## Creating traning pipeline

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb
import os
from typing import Dict, Any
import json
from tqdm import tqdm

class TitanTrainer:
    def __init__(
        self,
        model: nn.Module,
        tokenizer: CustomTokenizer,
        train_dataloader: torch.utils.data.DataLoader,
        val_dataloader: torch.utils.data.DataLoader = None,
        learning_rate: float = 1e-4,
        warmup_steps: int = 1000,
        max_steps: int = 100000,
        gradient_clip: float = 1.0,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu',
        checkpoint_dir: str = 'checkpoints',
        use_wandb: bool = False
    ):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.device = device
        self.checkpoint_dir = checkpoint_dir
        self.use_wandb = use_wandb

        # Create checkpoint directory
        os.makedirs(checkpoint_dir, exist_ok=True)

        # Initialize optimizer
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0.01
        )

        # Initialize learning rate scheduler
        self.scheduler = CosineAnnealingLR(
            self.optimizer,
            T_max=max_steps,
            eta_min=learning_rate/100
        )

        self.warmup_steps = warmup_steps
        self.gradient_clip = gradient_clip

        # Initialize training state
        self.current_step = 0
        self.best_val_loss = float('inf')

    def _get_warmup_lr(self, step: int) -> float:
        return min(1.0, step / self.warmup_steps)

    def _compute_loss(self, logits: torch.Tensor, targets: torch.Tensor, pad_idx: int) -> torch.Tensor:
        # Reshape logits and targets for loss computation
        logits = logits.view(-1, logits.size(-1))
        targets = targets.view(-1)

        # Create mask for padding
        mask = (targets != pad_idx).float()

        # Compute cross entropy loss
        loss = F.cross_entropy(logits, targets, reduction='none')

        # Apply mask and compute mean
        loss = (loss * mask).sum() / mask.sum()

        return loss

    def train_step(self, batch: Dict[str, torch.Tensor]) -> Dict[str, float]:
        self.model.train()

        # Move batch to device
        input_ids = batch['input_ids'].to(self.device)
        target_ids = batch['target_ids'].to(self.device)
        attention_mask = batch['attention_mask'].to(self.device)

        # Forward pass
        logits = self.model(input_ids, attention_mask)

        # Compute loss
        loss = self._compute_loss(logits, target_ids, self.tokenizer.pad_token_id)

        # Backward pass
        loss.backward()

        # Clip gradients
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.gradient_clip)

        # Apply warmup
        if self.current_step < self.warmup_steps:
            lr_mult = self._get_warmup_lr(self.current_step)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] *= lr_mult

        # Update parameters
        self.optimizer.step()
        self.scheduler.step()
        self.optimizer.zero_grad()

        return {'loss': loss.item()}

    # @torch.no_grad()
    # def evaluate(self) -> Dict[str, float]:
    #     if not self.val_dataloader:
    #         return {}

    #     self.model.eval()
    #     total_loss = 0
    #     total_tokens = 0

    #     for batch in self.val_dataloader:
    #         input_ids = batch['input_ids'].to(self.device)
    #         target_ids = batch['target_ids'].to(self.device)
    #         attention_mask = batch['attention_mask'].to(self.device)

    #         logits = self.model(input_ids, attention_mask)
    #         loss = self._compute_loss(logits, target_ids, self.tokenizer.pad_token_id)

    #         total_loss += loss.item() * input_ids.size(0)
    #         total_tokens += input_ids.size(0)

    #     return {'val_loss': total_loss / total_tokens}
    @torch.no_grad()
    def evaluate(self, dataloader: torch.utils.data.DataLoader) -> Dict[str, float]:
        """
        Evaluates the model on the provided dataloader.

        Args:
            dataloader: The dataloader to use for evaluation.

        Returns:
            A dictionary containing the evaluation metrics.
        """

        self.model.eval()
        total_loss = 0
        total_tokens = 0

        for batch in dataloader:  # Use the provided dataloader
            input_ids = batch['input_ids'].to(self.device)
            target_ids = batch['target_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)

            logits = self.model(input_ids, attention_mask)
            loss = self._compute_loss(logits, target_ids, self.tokenizer.pad_token_id)

            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)

        return {'val_loss': total_loss / total_tokens}

    def save_checkpoint(self, metrics: Dict[str, float], is_best: bool = False):
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'metrics': metrics,
            'step': self.current_step
        }

        # Save latest checkpoint
        path = os.path.join(self.checkpoint_dir, 'latest.pt')
        torch.save(checkpoint, path)

        # Save best checkpoint if needed
        if is_best:
            best_path = os.path.join(self.checkpoint_dir, 'best.pt')
            torch.save(checkpoint, best_path)




    def train(self, num_epochs: int):
        if self.use_wandb:
            wandb.init(project="titan-transformer")

        for epoch in range(num_epochs):
            epoch_metrics = {'epoch': epoch}

            # Training loop
            pbar = tqdm(self.train_dataloader, desc=f'Epoch {epoch}')
            for batch in pbar:
                metrics = self.train_step(batch)
                epoch_metrics.update(metrics)

                # Update progress bar
                pbar.set_postfix(loss=f"{metrics['loss']:.4f}")

                self.current_step += 1

                # Log metrics
                if self.use_wandb:
                    wandb.log(metrics, step=self.current_step)

            # Validation
            if self.val_dataloader:  # Check if a validation dataloader exists
                val_metrics = self.evaluate(self.val_dataloader)  # Pass the dataloader here
                epoch_metrics.update(val_metrics)

                # Check for best model
                if val_metrics and val_metrics['val_loss'] < self.best_val_loss:
                    self.best_val_loss = val_metrics['val_loss']
                    self.save_checkpoint(epoch_metrics, is_best=True)

            # Save regular checkpoint
            self.save_checkpoint(epoch_metrics)

            # Log epoch metrics
            if self.use_wandb:
                wandb.log(epoch_metrics, step=self.current_step)

            print(f"Epoch {epoch} metrics:", epoch_metrics)



    # def train(self, num_epochs: int):
    #     if self.use_wandb:
    #         wandb.init(project="titan-transformer")

    #     for epoch in range(num_epochs):
    #         epoch_metrics = {'epoch': epoch}

    #         # Training loop
    #         pbar = tqdm(self.train_dataloader, desc=f'Epoch {epoch}')
    #         for batch in pbar:
    #             metrics = self.train_step(batch)
    #             epoch_metrics.update(metrics)

    #             # Update progress bar
    #             pbar.set_postfix(loss=f"{metrics['loss']:.4f}")

    #             self.current_step += 1

    #             # Log metrics
    #             if self.use_wandb:
    #                 wandb.log(metrics, step=self.current_step)

    #         # Validation
    #         val_metrics = self.evaluate()
    #         epoch_metrics.update(val_metrics)

    #         # Check for best model
    #         if val_metrics and val_metrics['val_loss'] < self.best_val_loss:
    #             self.best_val_loss = val_metrics['val_loss']
    #             self.save_checkpoint(epoch_metrics, is_best=True)

    #         # Save regular checkpoint
    #         self.save_checkpoint(epoch_metrics)

    #         # Log epoch metrics
    #         if self.use_wandb:
    #             wandb.log(epoch_metrics, step=self.current_step)

    #         print(f"Epoch {epoch} metrics:", epoch_metrics)

    @torch.no_grad()
    def test(self, test_dataloader: torch.utils.data.DataLoader) -> Dict[str, float]:
        """
        Evaluates the model on the test dataset.

        Args:
            test_dataloader: The dataloader for the test dataset.

        Returns:
            A dictionary containing the evaluation metrics for the test dataset.
        """
        print("Evaluating on test dataset...")
        metrics = self.evaluate(test_dataloader)
        print(f"Test Results: {metrics}")
        return metrics



In [11]:
# train_texts = [
#     "The quick brown fox jumps over the lazy dog.",
#     "Artificial Intelligence is transforming the world.",
#     "Python is a popular programming language.",
#     "Machine learning requires large datasets for training.",
#     "The sky is clear and blue today.",
#     "Natural language processing enables machines to understand text.",
#     "Self-driving cars use advanced sensors and algorithms.",
#     "Data science combines statistics and computer science.",
#     "Reinforcement learning involves agents taking actions in environments.",
#     "Deep learning utilizes neural networks for predictions.",
#     "Quantum computing is a rapidly emerging field.",
#     "Cloud computing offers scalable storage solutions.",
#     "IoT devices are interconnected over networks.",
#     "Cybersecurity is crucial in today's digital era.",
#     "The sun rises in the east and sets in the west.",
#     "Renewable energy sources are vital for sustainability.",
#     "Blockchain technology ensures secure transactions.",
#     "Mathematics is the foundation of many sciences.",
#     "Artificial neural networks mimic the human brain.",
#     "The universe is vast and mysterious."
# ]
# val_texts = [
#     "Graphs are used to represent relationships between entities.",
#     "Programming requires logical thinking and problem-solving skills.",
#     "AI ethics is an important area of research.",
#     "The weather today is perfect for a picnic.",
#     "Virtual reality creates immersive digital experiences.",
#     "Big data analytics helps businesses make better decisions.",
#     "Space exploration expands our understanding of the cosmos.",
#     "Autonomous drones are used in agriculture and delivery.",
#     "Data visualization simplifies complex data insights.",
#     "The human genome holds the secrets of biology."
# ]
# test_texts = [
#     "Robotics integrates mechanics and AI for automation.",
#     "Digital transformation accelerates technological adoption.",
#     "Text-to-speech models improve accessibility.",
#     "The internet connects people across the globe.",
#     "Energy-efficient algorithms are vital for green computing."
# ]


## Datapreprocessing


### Load dataset

In [12]:
!pip install kagglehub

In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jsonali2003/linguogen-text-generation-dataset")

print("Path to dataset files:", path)



100%|██████████| 10.0k/10.0k [00:00<00:00, 20.6MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/jsonali2003/linguogen-text-generation-dataset/versions/1


In [14]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split

# Download the latest version of the dataset
path = kagglehub.dataset_download("jsonali2003/linguogen-text-generation-dataset")

print("Path to dataset files:", path)

# List all CSV files in the downloaded dataset folder
import os
csv_files = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.csv')]

# Combine all CSV files into one DataFrame
dataframes = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)

# Ensure the dataset has the necessary columns
assert 'Input Text' in combined_df.columns, "'Input Text' column is missing!"
assert 'Target Text' in combined_df.columns, "'Target Text' column is missing!"

# Split into input and target texts
input_texts = combined_df['Input Text'].tolist()
target_texts = combined_df['Target Text'].tolist()

# Split into train, validation, and test sets (70/15/15 split)
train_inputs, temp_inputs, train_targets, temp_targets = train_test_split(
    input_texts, target_texts, test_size=0.3, random_state=42
)

val_inputs, test_inputs, val_targets, test_targets = train_test_split(
    temp_inputs, temp_targets, test_size=0.5, random_state=42
)

# Combine inputs and targets for each set
train_texts = [text for sublist in zip(train_inputs, train_targets) for text in sublist]
val_texts = [text for sublist in zip(val_inputs, val_targets) for text in sublist]
test_texts = [text for sublist in zip(test_inputs, test_targets) for text in sublist]

print("Data processing completed!")
print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")


Path to dataset files: /root/.cache/kagglehub/datasets/jsonali2003/linguogen-text-generation-dataset/versions/1
Data processing completed!
Training set size: 712
Validation set size: 152
Test set size: 154


In [17]:
# Create dataloaders
train_dataloader = create_dataloader(train_texts, tokenizer, batch_size=32)
val_dataloader = create_dataloader(val_texts, tokenizer, batch_size=32)
test_dataloader = create_dataloader(test_texts, tokenizer, batch_size=32)

# Initialize trainer
# trainer = TitanTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataloader=train_dataloader,
#     val_dataloader=val_dataloader,
#     learning_rate=1e-4,
#     use_wandb=True  # Set to True if using Weights & Biases
# )

trainer = TitanTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    learning_rate=2e-6,  # Try a smaller learning rate
    use_wandb=True
)

# Train model



In [18]:
trainer.train(num_epochs=10)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 0: 100%|██████████| 23/23 [00:04<00:00,  5.10it/s, loss=1.6319]


...

Epoch 7 metrics: {'epoch': 7, 'loss': 1.6617921590805054, 'val_loss': 1.6298224925994873}


Epoch 8: 100%|██████████| 23/23 [00:04<00:00,  5.38it/s, loss=1.6483]


Epoch 8 metrics: {'epoch': 8, 'loss': 1.6482689380645752, 'val_loss': 1.6299194913161428}


Epoch 9: 100%|██████████| 23/23 [00:04<00:00,  5.33it/s, loss=1.6482]


Epoch 9 metrics: {'epoch': 9, 'loss': 1.648231029510498, 'val_loss': 1.6298907869740535}


In [19]:
trainer.train(num_epochs=20)


Epoch 0: 100%|██████████| 23/23 [00:04<00:00,  5.18it/s, loss=1.6268]


Epoch 0 metrics: {'epoch': 0, 'loss': 1.6268436908721924, 'val_loss': 1.6300630381232815}


Epoch 1: 100%|██████████| 23/23 [00:04<00:00,  5.30it/s, loss=1.6294]


Epoch 1 metrics: {'epoch': 1, 'loss': 1.6293824911117554, 'val_loss': 1.6306439889104742}


Epoch 2: 100%|██████████| 23/23 [00:04<00:00,  5.49it/s, loss=1.6311]


...

Epoch 17 metrics: {'epoch': 17, 'loss': 1.6477144956588745, 'val_loss': 1.6303890253368176}


Epoch 18: 100%|██████████| 23/23 [00:04<00:00,  5.45it/s, loss=1.6179]


Epoch 18 metrics: {'epoch': 18, 'loss': 1.6178721189498901, 'val_loss': 1.6301242991497642}


Epoch 19: 100%|██████████| 23/23 [00:04<00:00,  5.30it/s, loss=1.6513]


Epoch 19 metrics: {'epoch': 19, 'loss': 1.65126633644104, 'val_loss': 1.6295391383923983}


In [ ]:


test_results = trainer.test(test_dataloader)
print(f"Test Results: {test_results}")


## torch server handler class

In [20]:
# TorchServe handler class
class TitanHandler:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # def initialize(self, context):
    #     """Initialize model and tokenizer."""
    #     properties = context.system_properties
    #     model_dir = properties.get('model_dir')

    #     # Load tokenizer
    #     with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
    #         tokenizer_config = json.load(f)
    #     self.tokenizer = CustomTokenizer(**tokenizer_config)
    #     self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

    #     # Load model
    #     checkpoint = torch.load(
    #         os.path.join(model_dir, 'model.pt'),
    #         map_location=self.device
    #     )

    #     # Initialize model (you'll need to match the configuration used during training)
    #     self.model = TitanModelWithCustomEmbedding(
    #         embedding_layer=CustomEmbedding(
    #             vocab_size=self.tokenizer.vocab_size_current,
    #             embedding_dim=512  # Make sure this matches your training config
    #         )
    #     )
    #     self.model.load_state_dict(checkpoint['model_state_dict'])
    #     self.model.to(self.device)
    #     self.model.eval()
    # def initialize(self, context):
    #     """Initialize model and tokenizer."""
    #     properties = context.system_properties
    #     model_dir = properties.get('model_dir')

    #     # Load tokenizer
    #     with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
    #         tokenizer_config = json.load(f)
    #     self.tokenizer = CustomTokenizer(**tokenizer_config)
    #     self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

    #     # Load model
    #     checkpoint = torch.load(
    #         os.path.join(model_dir, 'model.pt'),
    #         map_location=self.device
    #     )

    #     # Initialize model with the correct vocabulary size from the loaded tokenizer
    #     self.model = TitanModelWithCustomEmbedding(
    #         embedding_layer=CustomEmbedding(
    #             vocab_size=self.tokenizer.vocab_size_current,  # Use tokenizer's vocab size
    #             embedding_dim=512  # Make sure this matches your training config
    #         ),
    #         max_seq_length=1024,  # Add this to match your training config
    #         depth=12           # Add this to match your training config
    #     )
    #     self.model.load_state_dict(checkpoint['model_state_dict'])
    #     self.model.to(self.device)
    #     self.model.eval()
    def initialize(self, context):
        """Initialize model and tokenizer."""
        properties = context.system_properties
        model_dir = properties.get('model_dir')

        # Load tokenizer FIRST
        with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
            tokenizer_config = json.load(f)
        self.tokenizer = CustomTokenizer(**tokenizer_config)
        self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

        # Print vocabulary size for debugging
        print(f"Loaded tokenizer vocabulary size: {self.tokenizer.vocab_size_current}")

        # Load checkpoint to check saved model configuration
        checkpoint = torch.load(
            os.path.join(model_dir, 'model.pt'),
            map_location=self.device
        )

        # Print model state dict shapes for debugging
        print("\nModel checkpoint shapes:")
        for name, param in checkpoint['model_state_dict'].items():
            print(f"{name}: {param.shape}")

        # Initialize model with the EXACT same vocabulary size as in checkpoint
        vocab_size = checkpoint['model_state_dict']['embedding.embedding.weight'].shape[0]
        print(f"\nInitializing model with vocabulary size: {vocab_size}")

        self.model = TitanModelWithCustomEmbedding(
            embedding_layer=CustomEmbedding(
                vocab_size=vocab_size,  # Use vocabulary size from checkpoint
                embedding_dim=512
            ),
            max_seq_length=1024,
            depth=12
        )

        # Load state dict
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()

    def preprocess(self, data):
        """Preprocess input text."""
        text = data[0].get('body').decode('utf-8')

        # Tokenize input
        input_ids = torch.tensor(
            [self.tokenizer.encode(text)],
            dtype=torch.long
        ).to(self.device)

        attention_mask = (input_ids != self.tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

    def inference(self, data):
        """Run inference on processed input."""
        with torch.no_grad():
            outputs = self.model(
                data['input_ids'],
                data['attention_mask']
            )
        return outputs

    def postprocess(self, inference_output):
        """Process model output to response format."""
        # Get predicted tokens
        predictions = inference_output.argmax(dim=-1)

        # Decode tokens to text
        response = self.tokenizer.decode(predictions[0].tolist())

        return [{'generated_text': response}]



In [23]:
# First, create a mock context class
import torch
import os
import json
class MockContext:
    def __init__(self, model_dir):
        self.system_properties = {
            'model_dir': model_dir  # Directory where model files are stored
        }

# Modified testing code
def test_model_generation(model_dir):
    """
    Test the model with various input prompts
    """
    # Create mock context
    context = MockContext(model_dir=model_dir)

    # Initialize handler with mock context
    handler = TitanHandler()
    handler.initialize(context)

    # Test cases
    test_cases = [
        # Simple completion tasks
        "The weather today is",
        "The cat sat on the",

        # Question answering
        "What is the capital of France?",
        "How does photosynthesis work?",

        # Environmental topics
        "Climate change is causing",
        "Renewable energy sources include",

        # Animal-related
        "Lions are known for",
        "Endangered species need protection because",
    ]

    print("Starting Model Testing...\n")
    print("-" * 50)

    for i, test_case in enumerate(test_cases, 1):
        print(f"\nTest Case {i}:")
        print(f"Input: {test_case}")

        try:
            # Process through the handler pipeline
            input_data = [{'body': test_case.encode('utf-8')}]
            processed = handler.preprocess(input_data)
            output = handler.inference(processed)
            result = handler.postprocess(output)

            print(f"Generated: {result[0]['generated_text']}")
            print("-" * 50)
        except Exception as e:
            print(f"Error processing test case: {str(e)}")
            print("-" * 50)

# Usage example:
model_dir = "/content/model_store"  # Replace with your model directory path
test_model_generation(model_dir)

Loaded tokenizer vocabulary size: 4


<ipython-input-20-9dd93b6c883c>:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(



Model checkpoint shapes:
pos_embedding: torch.Size([1, 1024, 512])
embedding.embedding.weight: torch.Size([6, 512])
transformer.layers.0.norm1.weight: torch.Size([512])
transformer.layers.0.norm1.bias: torch.Size([512])
transformer.layers.0.attn.q_proj.weight: torch.Size([512, 512])
transformer.layers.0.attn.q_proj.bias: torch.Size([512])
transformer.layers.0.attn.k_proj.weight: torch.Size([512, 512])
transformer.layers.0.attn.k_proj.bias: torch.Size([512])
transformer.layers.0.attn.v_proj.weight: torch.Size([512, 512])
transformer.layers.0.attn.v_proj.bias: torch.Size([512])
transformer.layers.0.attn.q_conv.depthwise.weight: torch.Size([512, 1, 3])
transformer.layers.0.attn.q_conv.depthwise.bias: torch.Size([512])
transformer.layers.0.attn.q_conv.pointwise.weight: torch.Size([512, 512, 1])
transformer.layers.0.attn.q_conv.pointwise.bias: torch.Size([512])
transformer.layers.0.attn.k_conv.depthwise.weight: torch.Size([512, 1, 3])
transformer.layers.0.attn.k_conv.depthwise.bias: torch.

In [24]:
import torch
from typing import List, Dict
import time

class TitanModelTester:
    def __init__(self, model_handler: TitanHandler):
        self.handler = model_handler

    def generate_text(self, prompt: str, max_length: int = 100) -> Dict:
        """
        Generate text from a single prompt and measure performance.
        """
        start_time = time.time()

        # Prepare input
        data = [{'body': prompt.encode('utf-8')}]

        # Run through handler pipeline
        processed = self.handler.preprocess(data)
        output = self.handler.inference(processed)
        response = self.handler.postprocess(output)

        end_time = time.time()

        return {
            'prompt': prompt,
            'generated_text': response[0]['generated_text'],
            'generation_time': end_time - start_time
        }

    def run_test_suite(self, temperature: float = 1.0) -> Dict[str, List[Dict]]:
        """
        Run a comprehensive suite of text generation tests.
        """
        test_cases = {
            'completion': [
                "Complete this sentence: The quick brown fox",
                "Write the next line: Roses are red,",
                "Finish this thought: If I could travel anywhere,"
            ],
            'creative': [
                "Write a short story about a magical library",
                "Create a poem about the ocean",
                "Describe a futuristic city"
            ],
            'technical': [
                "Explain how photosynthesis works",
                "Write a Python function that calculates fibonacci numbers",
                "Describe the process of machine learning"
            ],
            'dialogue': [
                "Write a conversation between two friends discussing their weekend",
                "Create a scene with dialogue between a teacher and student",
                "Write a job interview dialogue"
            ]
        }

        results = {}

        for category, prompts in test_cases.items():
            category_results = []
            for prompt in prompts:
                try:
                    result = self.generate_text(prompt)
                    result['status'] = 'success'
                except Exception as e:
                    result = {
                        'prompt': prompt,
                        'error': str(e),
                        'status': 'failed'
                    }
                category_results.append(result)
            results[category] = category_results

        return results

def format_test_results(results: Dict[str, List[Dict]]) -> str:
    """
    Format test results into a readable report.
    """
    report = []

    for category, tests in results.items():
        report.append(f"\n=== {category.upper()} TESTS ===\n")

        for i, test in enumerate(tests, 1):
            report.append(f"Test {i}:")
            report.append(f"Prompt: {test['prompt']}")

            if test['status'] == 'success':
                report.append(f"Generated Text: {test['generated_text']}")
                report.append(f"Generation Time: {test['generation_time']:.2f} seconds")
            else:
                report.append(f"Error: {test['error']}")

            report.append("")  # Empty line between tests

    return "\n".join(report)

# Example usage
if __name__ == "__main__":
    # Initialize the handler
    handler = TitanHandler()
    handler.initialize("write a poerty")  # You'll need to provide appropriate context

    # Create tester instance
    tester = TitanModelTester(handler)

    # Run test suite
    results = tester.run_test_suite()

    # Print formatted results
    print(format_test_results(results))

AttributeError: 'str' object has no attribute 'system_properties'

In [22]:
import os
import json
import torch


def prepare_model_for_serving(model_path: str, save_dir: str, tokenizer: CustomTokenizer):
    """
    Prepare model artifacts for TorchServe.

    Args:
        model_path (str): Path to the saved model checkpoint.
        save_dir (str): Directory to save the prepared model artifacts.
        tokenizer (CustomTokenizer): An instance of the tokenizer to save configuration and vocabulary.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Load the model from the checkpoint
    model = torch.load(model_path)

    # Save the model in the specified directory
    torch.save(model, os.path.join(save_dir, 'model.pt'))

    # Prepare tokenizer configuration
    tokenizer_config = {
        'vocab_size': tokenizer.vocab_size,
        'min_freq': tokenizer.min_freq,
        'special_tokens': tokenizer.special_tokens,
    }

    # Save tokenizer configuration as JSON
    with open(os.path.join(save_dir, 'tokenizer_config.json'), 'w') as f:
        json.dump(tokenizer_config, f)

    # Save tokenizer vocabulary
    tokenizer.save_vocab(os.path.join(save_dir, 'vocab.txt'))


# Example usage
# Create an instance of CustomTokenizer
tokenizer = CustomTokenizer(
    vocab_size=50000,
    min_freq=2,
    special_tokens=['<pad>', '<eos>', '<bos>', '<unk>']
)

# Call the function with the correct parameters
prepare_model_for_serving('checkpoints/best.pt', 'model_store', tokenizer)


<ipython-input-22-06d5ff2de07d>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [25]:
handler_code = """
# TorchServe handler class
class TitanHandler:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # def initialize(self, context):
    #
    #     properties = context.system_properties
    #     model_dir = properties.get('model_dir')

    #     # Load tokenizer
    #     with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
    #         tokenizer_config = json.load(f)
    #     self.tokenizer = CustomTokenizer(**tokenizer_config)
    #     self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

    #     # Load model
    #     checkpoint = torch.load(
    #         os.path.join(model_dir, 'model.pt'),
    #         map_location=self.device
    #     )

    #     # Initialize model (you'll need to match the configuration used during training)
    #     self.model = TitanModelWithCustomEmbedding(
    #         embedding_layer=CustomEmbedding(
    #             vocab_size=self.tokenizer.vocab_size_current,
    #             embedding_dim=512  # Make sure this matches your training config
    #         )
    #     )
    #     self.model.load_state_dict(checkpoint['model_state_dict'])
    #     self.model.to(self.device)
    #     self.model.eval()
    # def initialize(self, context):
    #
    #     properties = context.system_properties
    #     model_dir = properties.get('model_dir')

    #     # Load tokenizer
    #     with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
    #         tokenizer_config = json.load(f)
    #     self.tokenizer = CustomTokenizer(**tokenizer_config)
    #     self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

    #     # Load model
    #     checkpoint = torch.load(
    #         os.path.join(model_dir, 'model.pt'),
    #         map_location=self.device
    #     )

    #     # Initialize model with the correct vocabulary size from the loaded tokenizer
    #     self.model = TitanModelWithCustomEmbedding(
    #         embedding_layer=CustomEmbedding(
    #             vocab_size=self.tokenizer.vocab_size_current,  # Use tokenizer's vocab size
    #             embedding_dim=512  # Make sure this matches your training config
    #         ),
    #         max_seq_length=1024,  # Add this to match your training config
    #         depth=12           # Add this to match your training config
    #     )
    #     self.model.load_state_dict(checkpoint['model_state_dict'])
    #     self.model.to(self.device)
    #     self.model.eval()
    def initialize(self, context):

        properties = context.system_properties
        model_dir = properties.get('model_dir')

        # Load tokenizer FIRST
        with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
            tokenizer_config = json.load(f)
        self.tokenizer = CustomTokenizer(**tokenizer_config)
        self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

        # Print vocabulary size for debugging
        print(f"Loaded tokenizer vocabulary size: {self.tokenizer.vocab_size_current}")

        # Load checkpoint to check saved model configuration
        checkpoint = torch.load(
            os.path.join(model_dir, 'model.pt'),
            map_location=self.device
        )

        # Print model state dict shapes for debugging
        print("\nModel checkpoint shapes:")
        for name, param in checkpoint['model_state_dict'].items():
            print(f"{name}: {param.shape}")

        # Initialize model with the EXACT same vocabulary size as in checkpoint
        vocab_size = checkpoint['model_state_dict']['embedding.embedding.weight'].shape[0]
        print(f"\nInitializing model with vocabulary size: {vocab_size}")

        self.model = TitanModelWithCustomEmbedding(
            embedding_layer=CustomEmbedding(
                vocab_size=vocab_size,  # Use vocabulary size from checkpoint
                embedding_dim=512
            ),
            max_seq_length=1024,
            depth=12
        )

        # Load state dict
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()

    def preprocess(self, data):

        text = data[0].get('body').decode('utf-8')

        # Tokenize input
        input_ids = torch.tensor(
            [self.tokenizer.encode(text)],
            dtype=torch.long
        ).to(self.device)

        attention_mask = (input_ids != self.tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

    def inference(self, data):

        with torch.no_grad():
            outputs = self.model(
                data['input_ids'],
                data['attention_mask']
            )
        return outputs

    def postprocess(self, inference_output):

        # Get predicted tokens
        predictions = inference_output.argmax(dim=-1)

        # Decode tokens to text
        response = self.tokenizer.decode(predictions[0].tolist())

        return [{'generated_text': response}]



"""

# Save it to a file
with open("titan_handler.py", "w") as f:
    f.write(handler_code)

In [26]:
!pip install torchserve torch-model-archiver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 18.2 MB/s eta 0:00:00


In [34]:
!torch-model-archiver --model-name titan \
                      --version 1.0 \
                      --model-file model_store/model.pt \
                      --handler titan_handler.py \
                      --extra-files "model_store/tokenizer_config.json,model_store/vocab.txt" \
                      --export-path model_store \
                      --force


WARNING - Overwriting model_store/titan.mar ...


In [32]:
!lsof -i :8080


COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
node      7 root   21u  IPv6  21340      0t0  TCP *:8080 (LISTEN)
node      7 root   26u  IPv6 290112      0t0  TCP 049e73524b18:8080->172.28.0.1:49012 (ESTABLISHED)
node      7 root   28u  IPv6 308839      0t0  TCP 049e73524b18:8080->172.28.0.1:47122 (ESTABLISHED)
node      7 root   29u  IPv6 275399      0t0  TCP 049e73524b18:8080->172.28.0.1:34330 (ESTABLISHED)
node      7 root   31u  IPv6  41861      0t0  TCP 049e73524b18:8080->172.28.0.1:43008 (ESTABLISHED)


In [29]:
# Create a config.properties file
%%writefile config.properties
inference_address=http://127.0.0.1:8083
management_address=http://127.0.0.1:8084
metrics_address=http://127.0.0.1:8085
model_store=model_store
load_models=titan=titan.mar

Writing config.properties


In [35]:

# Start with minimal configuration
!torchserve --start --model-store model_store --models titan=titan.mar --disable-token-auth

Removing orphan pid file.
2025-01-20T18:07:55,019 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager -  Loading snapshot serializer plugin...
2025-01-20T18:07:55,170 [DEBUG] main org.pytorch.serve.util.ConfigManager - xpu-smi not available or failed: Cannot run program "xpu-smi": error=2, No such file or directory
2025-01-20T18:07:55,172 [WARN ] main org.pytorch.serve.util.ConfigManager - Your torchserve instance can access any URL to load models. When deploying to production, make sure to limit the set of allowed_urls in config.properties
2025-01-20T18:07:55,242 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2025-01-20T18:07:55,421 [INFO ] main org.pytorch.serve.metrics.configuration.MetricConfiguration - Successfully loaded metrics configuration from /usr/local/lib/python3.11/dist-packages/ts/configs/metrics.yaml
2025-01-20T18:07:55,545 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.12.0
TS Home: /usr/lo

In [34]:
!torchserve --start --model-store model_store --models titan=titan.mar


2025-01-20T17:19:27,325 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager -  Loading snapshot serializer plugin...
2025-01-20T17:19:27,519 [DEBUG] main org.pytorch.serve.util.ConfigManager - xpu-smi not available or failed: Cannot run program "xpu-smi": error=2, No such file or directory
2025-01-20T17:19:27,526 [WARN ] main org.pytorch.serve.util.ConfigManager - Your torchserve instance can access any URL to load models. When deploying to production, make sure to limit the set of allowed_urls in config.properties
2025-01-20T17:19:27,606 [INFO ] main org.pytorch.serve.util.TokenAuthorization - 
######
TorchServe now enforces token authorization by default.
This requires the correct token to be provided when calling an API.
Key file located at /content/key_file.json
Check token authorization documenation for information: https://github.com/pytorch/serve/blob/master/docs/token_authorization_api.md 
######

2025-01-20T17:19:27,606 [INFO ] main org.pytorch.serve.servingsdk.impl.

In [ ]:
!mkdir -p ~/.torchserve
!nano ~/.torchserve/config.properties


## push model to huggingface

In [36]:
from huggingface_hub import login

login()


In [37]:
from huggingface_hub import create_repo

create_repo("titan-transformer", private=False)  # Set `private=True` if you want the model to be private


HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-678e91d7-779f6a0a18d74ff153691531;f54ad6b9-0c1d-4f67-bdb6-c3a1b97e369f)

You already created this model repo

In [38]:
from huggingface_hub import upload_file

# Upload model.pt
upload_file(
    path_or_fileobj="model_store/model.pt",
    path_in_repo="pytorch_model.bin",  # This is the standard filename for the model weights
    repo_id="rajveer43/titan-transformer"  # Replace with your Hugging Face username and repo name
)

# Upload tokenizer_config.json
upload_file(
    path_or_fileobj="model_store/tokenizer_config.json",
    path_in_repo="tokenizer_config.json",
    repo_id="rajveer43/titan-transformer"
)

# Upload vocab.txt
upload_file(
    path_or_fileobj="model_store/vocab.txt",
    path_in_repo="vocab.txt",
    repo_id="rajveer43/titan-transformer"
)


model.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/rajveer43/titan-transformer/commit/4a0f6d070b5b64f388daa0a06581f17e806bbc63', commit_message='Upload vocab.txt with huggingface_hub', commit_description='', oid='4a0f6d070b5b64f388daa0a06581f17e806bbc63', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rajveer43/titan-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='rajveer43/titan-transformer'), pr_revision=None, pr_num=None)

In [39]:
from huggingface_hub import upload_file

# Upload model checkpoint
upload_file(
    path_or_fileobj="/content/checkpoints/best.pt",
    path_in_repo="model/checkpoints/best.pt",
    repo_id="rajveer43/titan-transformer",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/content/checkpoints/latest.pt",
    path_in_repo="model/checkpoints/latest.pt",
    repo_id="rajveer43/titan-transformer",
    repo_type="model"
)


best.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

latest.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rajveer43/titan-transformer/commit/7d02efea87c49703d56bec1fafe885af81ab1f6c', commit_message='Upload model/checkpoints/latest.pt with huggingface_hub', commit_description='', oid='7d02efea87c49703d56bec1fafe885af81ab1f6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rajveer43/titan-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='rajveer43/titan-transformer'), pr_revision=None, pr_num=None)

In [40]:
import os
from huggingface_hub import upload_file

logs_folder = '/content/logs'  # Path to your logs folder
repo_id = "rajveer43/titan-transformer"  # Your repo name on Hugging Face

# Loop through all files in the logs folder and upload them
for root, dirs, files in os.walk(logs_folder):
    for file in files:
        file_path = os.path.join(root, file)
        # Upload each file to the Hugging Face repo
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=f"logs/{file}",  # Keep the folder structure in the repo
            repo_id=repo_id,
            repo_type="model"
        )


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


In [42]:
import os
from huggingface_hub import upload_file

logs_folder = '/content/wandb'  # Path to your logs folder
repo_id = "rajveer43/titan-transformer"  # Your repo name on Hugging Face

# Loop through all files in the logs folder and upload them
for root, dirs, files in os.walk(logs_folder):
    for file in files:
        file_path = os.path.join(root, file)
        # Upload each file to the Hugging Face repo
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=f"wandb/{file}",  # Keep the folder structure in the repo
            repo_id=repo_id,
            repo_type="model"
        )


ValueError: Provided path: '/content/wandb/debug-internal.log' is not a file on the local file system